<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Swarm/swarm_project_psychotherapist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Briefing: AI-Driven Student Support System

## Objective
To develop an AI-driven system designed to provide personalized support and guidance for high school students through intelligent agents, leveraging APIs and natural language understanding. The system encompasses motivation, counseling, crisis handling, and therapy functionalities.

---

## Key Components

### 1. Technology Stack
- **Libraries and Tools:**
  - `swarm` (for agent orchestration)
  - `requests` (for API communication)
  - `BeautifulSoup` (for HTML parsing)
  - Google Colab's `userdata` (for secure API key management)
- **Agents:** Implemented using the Swarm library to manage specialized roles and interactions.

### 2. Agent Functionality
- **Motivator Agent:**
  - Retrieves stoic motivational quotes via an external API.
  - Provides motivational support by relating quotes to the student’s experience.
- **Counselor Agent:**
  - Offers academic and career guidance based on student data.
  - Ensures a supportive and engaging tone with concise recommendations.
- **Crisis Handler Agent:**
  - Supports students in distress by providing empathy and reassurance.
  - Facilitates a connection to a human counselor when necessary.
- **Therapist Agent:**
  - Engages with students to assess mental health based on mood history and academic data.
  - Dynamically routes students to the appropriate agent (Motivator, Counselor, or Crisis Handler) depending on their state.

### 3. Workflow
- User inputs are captured and passed through the Therapist Agent.
- Based on the student's context (e.g., mood, academic goals), the Therapist Agent decides the best-suited agent for interaction.
- Agents work collaboratively, transferring control as needed to maintain fluid and adaptive support.

### 4. Features
- Real-time interaction through terminal input and output.
- Dynamic message rendering using ANSI escape codes for color-coded and styled text (e.g., agent names in blue, tool calls in purple).
- Context-aware recommendations and actions.
- Integrated environment variable handling for secure API key access.

### 5. Contextual Data
- **Student Data:** Personal details, class, family situation.
- **Mood History:** Emotional patterns over time.
- **Academic Info:** GPA, professional goals, extracurriculars.

---

## Implementation Goals
- **Accuracy:** Tailored responses to individual student needs.
- **Empathy:** Foster trust and support through friendly, non-judgmental communication.
- **Scalability:** Modular design for expanding functionality or agent roles.
- **Interoperability:** Easy integration with other APIs and data sources.


## Setup

In [1]:
%pip install -qU git+https://github.com/openai/swarm.git requests

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 20.5 MB/s eta 0:00:00


In [2]:
# Set the API keys as environment variables
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
from swarm import Swarm, Agent
import requests
from IPython.display import Markdown
import json
from bs4 import BeautifulSoup

## Motivator agent

In [4]:
def get_motivational_quotes():
  return requests.get('https://stoic.tekloon.net/stoic-quote').json()

In [5]:
motivator_agent = Agent(
    name="Motivator Agent",
    instructions="Motivate student with stoic motivational quotes, present the quotes and relate it to student's experience.",
    functions=[get_motivational_quotes],
)

In [6]:
def transfer_to_motivator():
    return motivator_agent

## Counselor agent

In [8]:
def counselor(context_variables):
    # Retrieve student's academic information
    students_academic = context_variables.get("academic_info", None)

    # Return detailed, friendly, and approachable instructions
    return f"""
    You are a high school counselor specializing in helping students improve their academic performance
    and achieve admission to their dream universities.

    provide personalized advice and guidance tailored to their goals.

    Your approach should be:
    - Friendly and supportive, like a trusted advisor and mentor.
    - Focused on giving concise, actionable recommendations.
    - Avoid overwhelming the student with a long list of tasks; instead, make it short and clear.

    Make the conversation feel approachable and engaging, ensuring the student feels encouraged and motivated.
    """

In [9]:
counselor_agent = Agent(
    name="Counselor Agent",
    instructions=counselor,
)

In [10]:
def transfer_to_counselor():
    return counselor_agent

## Crisis handler agent


In [11]:
def crisis_handler():
    # Return clear, friendly, and supportive instructions
    return """
    You are a high school counselor trained in managing crisis situations.
    Your primary goal is to provide reassurance and support to the student while facilitating a connection to a human counselor.

    Here's what you should do:
    - Calmly and empathetically calm the student.
    - Reassure the student that help is on the way, and a human counselor will be with them shortly.
    - Maintain a warm, non-judgmental, and supportive tone throughout the interaction.

    Remember, your role is to provide comfort and instill a sense of safety until the human counselor arrives.
    """

In [12]:
crisis_handler_agent = Agent(
    name="Crisis Handler Agent",
    instructions=crisis_handler,
)

In [13]:
def transfer_to_crisis_handler():
    return crisis_handler

## Therapist agent

In [14]:
def therapist(context_variables):
  students_info = context_variables.get("student_data", None)
  students_mood_history = context_variables.get("mood_history", None)
  return f"""
    You are a therapist specializing in supporting high school students. Current Student: {students_info}
    Your primary tasks is to consider their past mood pattern {students_mood_history} and engage in conversations to understand their current mental state.
    If the student is feeling down or unmotivated, call the motivator agent.
    If the student is focusing on their academic goals or their future education, call the counselor agent.
    If the user is feeling bad and you feel a human attention is required, call the crisis handler agent.
    Your approach should be:
    Be friendly, approachable, and encouraging. Avoid giving a long list and keep the conversation natural.
    Call one agent at a time depending on the student's mood.
    """

In [15]:
# Main Agent
therapist_agent = Agent(
    name="Therapist Agent",
    instructions=therapist,
    functions = [transfer_to_motivator, transfer_to_counselor, transfer_to_crisis_handler]
)

In [16]:
def transfer_back_to_therapist():
    return therapist_agent

In [ ]:
motivator_agent.functions.append(transfer_back_to_therapist)
counselor_agent.functions.append(transfer_back_to_therapist)
crisis_handler_agent.functions.append(transfer_back_to_therapist)

## Conversation

In [17]:
def pretty_print_messages(messages) -> None:
    # Iterate over the list of messages
    for message in messages:
        # Skip messages that are not from the assistant
        if message["role"] != "assistant":
            continue

        # Print the sender's name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # If the message contains content, display it as Markdown
        if message["content"]:
            display(Markdown((message["content"])))

        # Extract tool calls, if any, from the message
        tool_calls = message.get("tool_calls") or []

        # Add a blank line if there are multiple tool calls
        if len(tool_calls) > 1:
            print()

        # Iterate through each tool call and print its details
        for tool_call in tool_calls:
            f = tool_call["function"]

            # Extract function name and arguments
            name, args = f["name"], f["arguments"]

            # Format arguments for display, replacing ":" with "=" for better readability
            arg_str = json.dumps(json.loads(args)).replace(":", "=")

            # Print the tool call in purple
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


In [18]:
context_variables = {
    "student_data": """Here is the internal data of the student:
        1. NAME: Salman Rehman
        2. CLASS: 10-A
        3. NOTES: problem with family
        """,
    "mood_history": ['happy', 'happy', 'neutral', 'sad', 'depressed', 'neutral', 'happy', 'sad'],
    "academic_info": """
        1. CLUB: Rugby
        2. PROFESSIONAL GOAL: Biologist
        3. VOLUNTEER: None
        4. GPA: 3.1
    """
}

In [19]:
messages = []

client = Swarm()
while True:
  user_input = input("\033[90mUser\033[0m: ")

  messages.append({"role": "user", "content": user_input})

  if user_input.lower() == 'bye':
    break

  response = client.run(
      agent=therapist_agent,
      messages=messages,
      context_variables=context_variables or {},
  )

  pretty_print_messages(response.messages)

  messages.extend(response.messages)
  agent = response.agent

User: hello 
Therapist Agent: 

Hello Salman! How are you feeling today?

User: i am tired 
Therapist Agent: 

I'm sorry to hear that you're feeling tired. Is it related to your studies, family, or something else? Would you like to talk about it?

User: i dont have any motivation to study
Therapist Agent: transfer_to_motivator()
Motivator Agent: get_motivational_quotes()
Motivator Agent: 

...

User:  i dont know
Therapist Agent: 

That's completely okay. Sometimes, just taking the time to reflect and speak about what's on your mind can be a helpful start. 

Would you like to try breaking down your tasks into smaller, more manageable pieces, or is there something specific on your mind about your studies or family that might be causing this lack of motivation? We can look into possible strategies together.

User: I am tired of living
Therapist Agent: transfer_to_crisis_handler()
Therapist Agent: 

It's very important to talk to someone who can help. Please reach out to a trusted adult, a school counselor, or a mental health professional immediately.

Remember, you're not alone, and there are people ready to support you through this.

User: bye
